<img src='../../images/besm.png' width='150px'>
<style>
    
@font-face {font-family: "B Lotus"; src: url("//db.onlinewebfonts.com/t/1605a655ba0a3246ce5eca3eaff6c5c2.eot"); src: url("//db.onlinewebfonts.com/t/1605a655ba0a3246ce5eca3eaff6c5c2.eot?#iefix") format("embedded-opentype"), url("//db.onlinewebfonts.com/t/1605a655ba0a3246ce5eca3eaff6c5c2.woff2") format("woff2"), url("//db.onlinewebfonts.com/t/1605a655ba0a3246ce5eca3eaff6c5c2.woff") format("woff"), url("//db.onlinewebfonts.com/t/1605a655ba0a3246ce5eca3eaff6c5c2.ttf") format("truetype"), url("//db.onlinewebfonts.com/t/1605a655ba0a3246ce5eca3eaff6c5c2.svg#B Lotus") format("svg"); }


</style>

<center style ='font-family: "B Lotus";'>بازیابی پیشرفته اطلاعات - درس پردازش زبان‌های طبیعی </center>
<center style ='font-family: "B Lotus";'> آزمایشگاه پردازش هوشمند متن و زبان و علوم انسانی محاسباتی </center>
<br>
<center> http://language.ml </center>
<center> contact: ehsan.asgari [AT] sharif [dot] edu </center>
<center> semesters: Fall 2021 / Spring 2022 (Updated) </center>

<h2 style='direction:rtl;'> § نمونه‌ای از متن  </h2> 



In [3]:
import _pickle as pickle

class RomanizeUtility(object):

    def __init__(self):
        self.arabic2roman = RomanizeUtility.load_obj(F'../../exploring-datasets/religious_text/quranic_data/data_config/arabic2roman.pickle')
        self.roman2arabic = RomanizeUtility.load_obj(F'../../exploring-datasets/religious_text/quranic_data/data_config/roman2arabic.pickle')

    @staticmethod
    def load_obj(filename):
        return pickle.load(open(filename, "rb"))
        
    def get_arabic_token(self, token):
        '''
        :param token:
        :return:
        '''
        return ''.join([self.roman2arabic[e1] for e1 in list(token) if e1 in self.roman2arabic])

    def get_romanize_token(self, token):
        '''
        :param token:
        :return:
        '''
        return ''.join([self.arabic2roman[e1] for e1 in list(token) if e1 in self.arabic2roman])


    def get_arabic_text(self, text):
        '''
        :param text:
        :return:
        '''
        return  ' '.join([self.get_arabic_token(x) for x in text.split()])


In [6]:
import random 
import nltk
import pandas as pd
import codecs
import tqdm

quranic_dir = "../../exploring-datasets/religious_text/quranic_data"
df = pd.read_csv(F'{quranic_dir}/id_text_with_orthographies.txt', sep='\t', header=None)
verse_complete_dict = pd.Series(df[1].tolist(), index=df[0]).to_dict()


# ROOT/LEMMA
ru=RomanizeUtility()
verse_lemma_dict = {line.split()[1]:ru.get_arabic_text(' '.join(line.split()[2::])) for line in [l.strip() for l in codecs.open(F'{quranic_dir}/id_quran_lem.txt','r','utf-8').readlines()]}
verse_root_dict = {line.split()[1]:ru.get_arabic_text(' '.join(line.split()[2::])) for line in [l.strip() for l in codecs.open(F'{quranic_dir}/id_quran_roots.txt','r','utf-8').readlines()]}

In [7]:
verse_complete_dict['2##186']

'وَإِذَا سَأَلَكَ عِبَادِي عَنِّي فَإِنِّي قَرِيبٌ ۖ أُجِيبُ دَعْوَةَ الدَّاعِ إِذَا دَعَانِ ۖ فَلْيَسْتَجِيبُوا لِي وَلْيُؤْمِنُوا بِي لَعَلَّهُمْ يَرْشُدُونَ'

In [8]:
verse_lemma_dict['2##186']

'إِذَا سَأَلَ عَبْد عَن إِنّ قَرِیب أُجِیبَت دَعْوَة دَّاع إِذَا دَعَا ٱسْتَجَابَ ءَامَنَ لَعَلّ یَرْشُدُ'

In [9]:
verse_root_dict['2##186']

'سال عبد قرب جوب دعو دعو دعو جوب امن رشد'

<h2 style='direction:rtl;'> § نرمالایز کردن  </h2> 



In [13]:
#!pip install camel_tools

In [14]:
from camel_tools.utils.normalize import normalize_unicode
from camel_tools.utils.normalize import normalize_alef_maksura_ar
from camel_tools.utils.normalize import normalize_alef_ar

from camel_tools.utils.normalize import normalize_alef_bw
from camel_tools.utils.normalize import normalize_alef_hsb

from camel_tools.utils.normalize import normalize_teh_marbuta_ar
from camel_tools.utils.dediac import dediac_ar

def normalize_arabic(sentence):

    # Normalize alef variants to 'ا'
    sent_norm = normalize_unicode(sentence)
    
    sent_norm = normalize_alef_bw(sent_norm)
    # Normalize alef variants to 'ا'
    sent_norm = normalize_alef_ar(sentence)

    # Normalize alef maksura 'ى' to yeh 'ي'
    sent_norm = normalize_alef_maksura_ar(sent_norm)

    # Normalize teh marbuta 'ة' to heh 'ه'
    sent_norm = normalize_teh_marbuta_ar(sent_norm)
    return dediac_ar(sent_norm)

In [15]:
from camel_tools.utils import normalize


verse_complete_dict_nrmlz = {k:normalize_arabic(v) for k,v in tqdm.tqdm(verse_complete_dict.items())}

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6236/6236 [00:00<00:00, 20130.22it/s]


In [18]:
verse_complete_dict['2##186']

'وَإِذَا سَأَلَكَ عِبَادِي عَنِّي فَإِنِّي قَرِيبٌ ۖ أُجِيبُ دَعْوَةَ الدَّاعِ إِذَا دَعَانِ ۖ فَلْيَسْتَجِيبُوا لِي وَلْيُؤْمِنُوا بِي لَعَلَّهُمْ يَرْشُدُونَ'

In [19]:
verse_complete_dict_nrmlz['2##186']

'واذا سالك عبادي عني فاني قريب ۖ اجيب دعوه الداع اذا دعان ۖ فليستجيبوا لي وليؤمنوا بي لعلهم يرشدون'

<h2 style='direction:rtl;'> § توکنایزیشن  </h2> 



In [20]:
quranic_lemma = [sents.split() for sents in tqdm.tqdm(verse_lemma_dict.values())]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6236/6236 [00:00<00:00, 293626.71it/s]


In [21]:
random.sample(quranic_lemma, 30)

[['إِذَا', 'تَلَيٰ', 'عَلَيٰ', 'ءَایَة', 'قَالَ', 'أَسَٰطِیر', 'أَوَّل'],
 ['وَجْه', 'یَوْمَئِذ', 'مُّسْفِرَة'],
 ['مَا',
  'فِي',
  'سَمَآء',
  'مَا',
  'فِي',
  'أَرْض',
  'مَا',
  'بَیْن',
  'مَا',
  'تَحْت',
  'ثَّرَيٰ'],
 ['لَا', 'سَمِعَ', 'فِي', 'لَٰغِیَة'],
 ['قَالَ',
  'ٱلَّذِي',
  'كَفَرَ',
  'هَل',
  'دَلَّ',
  'عَلَيٰ',
  'رَجُل',
  'نَبَّأَ',
  'إِذَا',
  'مُزِّقْ',
  'كُلّ',
  'مُمَزَّق',
  'إِنّ',
  'فِي',
  'خَلْق',
  'جَدِید'],
 ['حمٓ'],
 ['هَزَمُ',
  'إِذْن',
  'ٱللَّه',
  'قَتَلَ',
  'دَاوُد',
  'جَالُوت',
  'آتَي',
  'ٱللَّه',
  'مُلْك',
  'حِكْمَة',
  'عَلَّمَ',
  'مِن',
  'مَا',
  'شَآءَ',
  'لَوْلَآ',
  'دَفْع',
  'ٱللَّه',
  'نَّاس',
  'بَعْض',
  'بَعْض',
  'فَسَدَتِ',
  'أَرْض',
  'لَٰكِنّ',
  'ٱللَّه',
  'ذُو',
  'فَضْل',
  'عَلَيٰ',
  'عَٰلَمِین'],
 ['سَمَآء', 'مَا', 'بَنَيٰ'],
 ['ٱلَّذِي',
  'ٱتَّخَذَ',
  'دِین',
  'لَهْو',
  'لَعِب',
  'غَرَّ',
  'حَیَوٰة',
  'دُّنْیَا',
  'یَوْم',
  'نَسِيَ',
  'مَا',
  'نَسِيَ',
  'لِقَآء',
  'یَوْم',
  'هَٰذَا',
  'مَا',


<h2 style='direction:rtl;'> § تحلیل بسامد  </h2> 

In [22]:
from nltk import FreqDist
import itertools
import pandas as pd

lemma_tokens = list(itertools.chain(*[x.split() for x in verse_lemma_dict.values()]))
root_tokens = list(itertools.chain(*[x.split() for x in verse_root_dict.values()]))
original_tokens = list(itertools.chain(*[x.split() for x in verse_complete_dict_nrmlz.values()]))

dataframe = {}

for opt in ['lemma', 'root', 'original']:
     dataframe[opt] = FreqDist(eval(F"{opt}_tokens")).most_common(30)

freq_analysis = pd.DataFrame(dataframe)    

In [23]:
freq_analysis

,lemma,root,original
0,"(مِن, 3225)","(اله, 2851)","(من, 2763)"
1,"(ٱللَّه, 2699)","(قول, 1722)","(الله, 2155)"
2,"(مَا, 2565)","(كون, 1390)","(ۚ, 1972)"
3,"(لَا, 1738)","(ربب, 980)","(ۖ, 1682)"
4,"(فِي, 1701)","(امن, 879)","(ان, 1605)"
5,"(إِنّ, 1682)","(علم, 854)","(في, 1185)"
6,"(قَالَ, 1618)","(قوم, 660)","(ما, 1010)"
7,"(ٱلَّذِي, 1464)","(اتی, 549)","(لا, 812)"
8,"(عَلَيٰ, 1445)","(كفر, 525)","(الذين, 810)"
9,"(كَانَ, 1358)","(بین, 523)","(الا, 763)"


<h2 style='direction:rtl;'> § حذف stop-word  </h2> 

In [24]:
stopwords = [x.strip() for x in codecs.open(F'{quranic_dir}/quranic_stopwords.txt','r','utf-8').readlines()]
stopwords = stopwords + [normalize_arabic(x.strip()) for x in codecs.open(F'{quranic_dir}/quranic_stopwords.txt','r','utf-8').readlines()]

In [25]:
lemma_tokens_nonstop = [t for t in tqdm.tqdm(lemma_tokens) if t not in stopwords]
root_tokens_nonstop = [t for t in tqdm.tqdm(root_tokens) if t not in stopwords]
original_tokens_nonstop = [t for t in tqdm.tqdm(original_tokens) if t not in stopwords]

dataframe_nonstop = {}

for opt in ['lemma', 'root', 'original']:
     dataframe_nonstop[opt] = FreqDist(eval(F"{opt}_tokens_nonstop")).most_common(50)

freq_analysis_nonstop = pd.DataFrame(dataframe_nonstop)    

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 82176/82176 [00:00<00:00, 410877.94it/s]


<h4 style='direction:rtl;'> برای داده خود لیست stop-word ها را بهبود بدهید </h4> 

In [26]:
freq_analysis_nonstop

,lemma,root,original
0,"(ٱللَّه, 2699)","(اله, 2851)","(الله, 2155)"
1,"(قَالَ, 1618)","(قول, 1722)","(ۚ, 1972)"
2,"(ٱلَّذِي, 1464)","(كون, 1390)","(ۖ, 1682)"
3,"(كَانَ, 1358)","(ربب, 980)","(الذين, 810)"
4,"(رَبّ, 975)","(امن, 879)","(ولا, 658)"
5,"(إِلَّا, 663)","(علم, 854)","(وما, 646)"
6,"(ءَامَنَ, 537)","(قوم, 660)","(ۗ, 603)"
7,"(أَرْض, 461)","(اتی, 549)","(قال, 416)"
8,"(یَوْم, 405)","(كفر, 525)","(لهم, 373)"
9,"(قَوْم, 383)","(شیا, 519)","(يا, 350)"


<h2 style='direction:rtl;'> § استفاده از POS-tags  </h2> 

In [28]:
from camel_tools.utils.charmap import CharMapper


bw2ar = CharMapper.builtin_mapper('bw2ar')


df = pd.read_csv(F'{quranic_dir}/morphology_0.62.txt', delimiter='\t')

In [29]:
df

,LOCATION,FORM,WORD,TAG,FEATURES
0,(1:1:1:1),bi,bi,P,PREFIX|bi+
1,(1:1:1:2),somi,somi,N,STEM|POS:N|LEM:{som|ROOT:smw|M|GEN
2,(1:1:2:1),{ll~ahi,{ll~ahi,PN,STEM|POS:PN|LEM:{ll~ah|ROOT:Alh|GEN
3,(1:1:3:1),{l,{l,DET,PREFIX|Al+
4,(1:1:3:2),r~aHoma`ni,r~aHoma`ni,ADJ,STEM|POS:ADJ|LEM:r~aHoma`n|ROOT:rHm|MS|GEN
...,...,...,...,...,...
128214,(114:6:2:1),{lo,{lo,DET,PREFIX|Al+
128215,(114:6:2:2),jin~api,jin~api,N,STEM|POS:N|LEM:jin~ap|ROOT:jnn|F|GEN
128216,(114:6:3:1),wa,wa,CONJ,PREFIX|w:CONJ+
128217,(114:6:3:2),{l,{l,DET,PREFIX|Al+


In [30]:
df['Form-arabic']=df['FORM'].apply(lambda x: bw2ar(str(x)) )
df['Verse']=df['LOCATION'].apply(lambda x: '##'.join(x[1:-1].split(':')[0:2]) )

In [31]:
from collections import OrderedDict
ayat_dict=OrderedDict()
verse_IDs = df.Verse.tolist()
current_ID = verse_IDs[0]
ayat_dict[current_ID] = []

for idx, pos_tup in tqdm.tqdm(enumerate(list(zip(df['Form-arabic'].tolist(),df.TAG.tolist())))):
    if verse_IDs[idx]==current_ID:
        ayat_dict[current_ID].append(pos_tup)
    else:
        current_ID = verse_IDs[idx]
        ayat_dict[current_ID] = [pos_tup]

128219it [00:00, 1719952.11it/s]


In [32]:
all_verses = list(ayat_dict.values())
all_verses[0]

[('بِ', 'P'),
 ('سْمِ', 'N'),
 ('ٱللَّهِ', 'PN'),
 ('ٱل', 'DET'),
 ('رَّحْمَٰنِ', 'ADJ'),
 ('ٱل', 'DET'),
 ('رَّحِيمِ', 'ADJ')]

In [33]:
def vasf_ext(sentence):
    grammar = r"""
      SEFAT: {<PN><DET>?<ADJ>}
    """
    cp = nltk.RegexpParser(grammar)
    return (cp.parse(sentence))

In [34]:
mosufs = []
for sentence in tqdm.tqdm(all_verses):
    tree=vasf_ext(sentence)
    for subtree in tree.subtrees():
        if subtree.label() == 'SEFAT':
            mosufs.append(subtree.leaves())

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6236/6236 [00:00<00:00, 14380.20it/s]


In [35]:
mosufs

[[('ٱللَّهِ', 'PN'), ('ٱل', 'DET'), ('رَّحْمَٰنِ', 'ADJ')],
 [('ٱللَّهِ', 'PN'), ('مُصَدِّقٌ', 'ADJ')],
 [('ٱللَّهِ', 'PN'), ('مُصَدِّقٌ', 'ADJ')],
 [('شَّيْطَٰنِ', 'PN'), ('ٱل', 'DET'), ('رَّجِيمِ', 'ADJ')],
 [('ٱللَّهِ', 'PN'), ('ٱلْ', 'DET'), ('عَزِيزِ', 'ADJ')],
 [('ٱللَّهِ', 'PN'), ('كَثِيرًا', 'ADJ')],
 [('ٱللَّهَ', 'PN'), ('كَثِيرًا', 'ADJ')],
 [('ٱللَّهَ', 'PN'), ('بَرِى^ءٌ', 'ADJ')],
 [('ٱللَّهَ', 'PN'), ('غَفُورٌ', 'ADJ')],
 [('ٱللَّهُ', 'PN'), ('أَحَقُّ', 'ADJ')],
 [('ٱللَّهُ', 'PN'), ('عَلِيمٌ', 'ADJ')],
 [('ٱللَّهُ', 'PN'), ('خَبِيرٌ[', 'ADJ')],
 [('ٱللَّهُ', 'PN'), ('غَفُورٌ', 'ADJ')],
 [('ٱللَّهَ', 'PN'), ('عَلِيمٌ', 'ADJ')],
 [('ٱللَّهُ', 'PN'), ('عَزِيزٌ', 'ADJ')],
 [('ٱللَّهُ', 'PN'), ('عَلِيمٌ[', 'ADJ')],
 [('ٱللَّهُ', 'PN'), ('عَلِيمٌ[', 'ADJ')],
 [('ٱللَّهُ', 'PN'), ('عَلِيمٌ', 'ADJ')],
 [('ٱللَّهَ', 'PN'), ('عَزِيزٌ', 'ADJ')],
 [('ٱللَّهِ', 'PN'), ('أَكْبَرُ', 'ADJ')],
 [('ٱللَّهَ', 'PN'), ('عَلَّٰمُ', 'ADJ')],
 [('جَهَنَّمَ', 'PN'), ('أَشَدُّ', 'ADJ')],
 [('ٱللَّ

In [36]:
mosuf_texts = [' '.join([y[0] for y in x]) for x in mosufs]


nltk.FreqDist(mosuf_texts).most_common(50)

[('ٱللَّهُ عَلِيمٌ', 9),
 ('ٱللَّهَ غَفُورٌ', 8),
 ('ٱللَّهِ ٱلْ عَزِيزِ', 6),
 ('ٱللَّهَ كَثِيرًا', 5),
 ('ٱللَّهُ أَعْلَمُ', 5),
 ('ٱللَّهِ ٱلْ مُخْلَصِينَ', 5),
 ('ٱللَّهُ غَفُورٌ', 4),
 ('ٱللَّهَ عَلِيمٌ', 4),
 ('ٱللَّهُ عَلِيمٌ[', 4),
 ('ٱللَّهَ عَزِيزٌ', 4),
 ('ٱللَّهَ عَلِيمٌ[', 4),
 ('ٱللَّهِ خَيْرٌ', 4),
 ('قُرْءَانًا عَرَبِيًّا', 4),
 ('ٱللَّهَ سَمِيعٌ[', 4),
 ('ٱللَّهِ يَسِيرٌ', 4),
 ('ٱللَّهُ غَفُورًا', 4),
 ('ٱللَّهِ أَكْبَرُ', 3),
 ('ٱللَّهُ سَمِيعٌ', 3),
 ('ٱللَّهِ شَهِيدً[ا', 3),
 ('ٱللَّهُ ٱلْ وَٰحِدُ', 3),
 ('ٱللَّهُ خَيْرٌ', 3),
 ('ٱللَّهُ ٱلْ عَزِيزُ', 3),
 ('ٱللَّهِ ٱل رَّحْمَٰنِ', 2),
 ('ٱللَّهِ مُصَدِّقٌ', 2),
 ('شَّيْطَٰنِ ٱل رَّجِيمِ', 2),
 ('ٱللَّهِ كَثِيرًا', 2),
 ('ٱللَّهُ أَحَقُّ', 2),
 ('ٱللَّهُ خَبِيرٌ[', 2),
 ('قُرْءَٰنًا عَرَبِيًّا', 2),
 ('لَّهِ ٱلْ وَٰحِدِ', 2),
 ('ٱللَّهَ لَطِيفٌ', 2),
 ('ٱللَّهَ خَبِيرٌ[', 2),
 ('ٱللَّهَ غَنِىٌّ', 2),
 ('ٱللَّهِ يَسِيرًا', 2),
 ('ٱللَّهُ عَزِيزًا', 2),
 ('ٱللَّهَ قَوِىٌّ', 2),
 ('ٱللَّهَ بَرِى^ءٌ', 1),
 ('ٱللَّهُ عَ

<h2 style='direction:rtl;'> § دیدن سیاق  </h2> 

In [37]:
len(all_verses)

6236

In [38]:
ayah_text =[normalize_arabic(sent) for sent in list(itertools.chain(*[[x for x,y in verse] for verse in all_verses]))]

ayah_text = nltk.Text(ayah_text)


In [39]:
ayah_text[0:10]

['ب', 'سم', 'الله', 'ال', 'رحمن', 'ال', 'رحيم', 'ال', 'حمد', 'ل']

In [40]:
ayah_text.concordance('روح')

Displaying 23 of 23 matches:
 نا عيسي ابن مريم ال بينت و ايد ن ه ب روح ال قدس ا ف كلما جا^ء كم رسول[ ب ما لا
 نا عيسي ابن مريم ال بينت و ايد ن ه ب روح ال قدس و لو شا^ء الله ما اقتتل الذين 
ل الله و كلمت ه,^ القي ها^ الي مريم و روح من ه ف #امن وا@ ب الله و رسل ه. و لا 
عمت ي علي ك و علي ولدت ك اذ ايد ت ك ب روح ال قدس تكلم ال ناس في ال مهد و كهلا و
ا@ من يوسف و اخي ه و لا تا@ي#س وا@ من روح الله ان ه, لا يا@ي#س من روح الله الا 
#س وا@ من روح الله ان ه, لا يا@ي#س من روح الله الا ال قوم ال كفرون ف لما دخل وا
 مسنون ف اذا سوي ت ه, و نفخ ت في ه من روح ي ف قع وا@ ل ه, سجدين ف سجد ال مل^ئكه
تعلي ع ما يشرك ون ينزل ال مل^ئكه ب ال روح من امر ه. علي من يشا^ء من عباد ه.^ ان
مفتر] بل اكثر هم لا يعلم ون قل نزل ه, روح ال قدس من رب ك ب ال حق ل يثبت الذين ء
 ب من هو اهدي سبيلا و يس#ل ون ك عن ال روح قل ال روح من امر رب ي و ما^ اوتي تم م
هدي سبيلا و يس#ل ون ك عن ال روح قل ال روح من امر رب ي و ما^ اوتي تم من ال علم ا
خذت من دون هم حجابا ف ارسل نا^ الي ها روح نا ف تمثل ل ها بشرا سويا قالت ان ي^ ا
 التي^ احصن